# Модуль 2.2 — RouterChain (маршрутизация запросов)

**Цель:** научиться направлять запросы в разные цепочки в зависимости от темы.

**Что сделаем:**
- установим библиотеки
- настроим ключ и base URL
- построим простой роутер (поддержка vs продажи)
- разберем параметры и практические советы

In [ ]:
%pip -q install -U \
  langchain \
  langchain-openai \
  langchain-community \
  pydantic==2.12.3 \
  python-dotenv \
  tiktoken \
  requests==2.32.4

## Настройка ключа и base URL

Для AITunnel укажите `OPENAI_API_KEY` и `OPENAI_BASE_URL`. В Colab лучше хранить ключ в переменной окружения.

> Подсказка: можно задать ключ один раз в разделе **Secrets** (значок 🔑 слева в Colab), тогда он будет доступен во всех ячейках.

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Введите OPENAI_API_KEY: ")

if not os.environ.get("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

## Идея роутера

Мы принимаем пользовательский запрос и направляем его в правильную цепочку:
- **Поддержка** — вопросы про ошибки, доставку, возвраты.
- **Продажи** — вопросы про тарифы, скидки, возможности.

Роутер решает, куда отправить запрос, и это снижает стоимость и повышает качество ответа.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

base_url = os.environ.get("OPENAI_BASE_URL")
chat = ChatOpenAI(
    model="gpt-5.2-chat",
    temperature=0.2,
    max_tokens=256,
    base_url=base_url,
)

support_prompt = PromptTemplate(
    input_variables=["text"],
    template=(
        "Ты сотрудник поддержки. Ответь кратко и по делу.\n"
        "Запрос: {text}"
    ),
)

sales_prompt = PromptTemplate(
    input_variables=["text"],
    template=(
        "Ты менеджер по продажам. Ответь дружелюбно и предложи следующий шаг.\n"
        "Запрос: {text}"
    ),
)

fallback_prompt = PromptTemplate(
    input_variables=["text"],
    template=(
        "Ты ассистент. Дай нейтральный ответ и уточни запрос.\n"
        "Запрос: {text}"
    ),
)

support_chain = support_prompt | chat | StrOutputParser()
sales_chain = sales_prompt | chat | StrOutputParser()
fallback_chain = fallback_prompt | chat | StrOutputParser()

# Простая логика маршрутизации по ключевым словам
router = RunnableBranch(
    (
        lambda x: any(k in x.lower() for k in ["ошибка", "доставка", "возврат", "не работает"]),
        support_chain,
    ),
    (
        lambda x: any(k in x.lower() for k in ["цена", "тариф", "скидка", "купить"]),
        sales_chain,
    ),
    fallback_chain,
)

queries = [
    "У меня ошибка при оплате, что делать?",
    "Какие есть тарифы и скидки?",
    "Подскажите, как это работает?",
]

for q in queries:
    print("\nЗапрос:", q)
    print("Ответ:", router.invoke(q))

## Разбор и рекомендации

**Что важно:**
- Роутер выбирает цепочку **до вызова модели**, поэтому экономит деньги.
- Ключевые слова — быстрый способ, но не самый точный.

**Практика:**
- Для продакшена используйте классификацию через модель (router‑LLM), если ключевые слова не хватает.
- Логируйте, куда попал запрос, чтобы улучшать правила.
- Делайте `fallback`‑цепочку на случай неоднозначных запросов.